# Zero to Snowflake - Simple Data Pipeline

## トピック
1. 外部ステージからの取り込み
2. 半構造化データとVARIANTデータ型
3. 動的テーブル
4. 動的テーブルによるシンプルなパイプライン
5. 有向非循環グラフ（DAG）によるパイプライン可視化

## コンテキスト設定

In [ ]:
%%sql
USE DATABASE tb_101;
USE ROLE tb_data_engineer;
USE WAREHOUSE tb_de_wh;

## 1. 外部ステージからの取り込み

In [ ]:
%%sql
-- menu_stage ステージを作成
CREATE OR REPLACE STAGE raw_pos.menu_stage
COMMENT = 'メニューデータ用ステージ'
URL = 's3://sfquickstarts/frostbyte_tastybytes/raw_pos/menu/'
FILE_FORMAT = public.csv_ff;

In [ ]:
%%sql
CREATE OR REPLACE TABLE raw_pos.menu_staging
(
    menu_id NUMBER(19,0),
    menu_type_id NUMBER(38,0),
    menu_type VARCHAR(16777216),
    truck_brand_name VARCHAR(16777216),
    menu_item_id NUMBER(38,0),
    menu_item_name VARCHAR(16777216),
    item_category VARCHAR(16777216),
    item_subcategory VARCHAR(16777216),
    cost_of_goods_usd NUMBER(38,4),
    sale_price_usd NUMBER(38,4),
    menu_item_health_metrics_obj VARIANT
);

In [ ]:
%%sql
-- データをロード
COPY INTO raw_pos.menu_staging
FROM @raw_pos.menu_stage;

In [ ]:
%%sql
-- ロードの確認
SELECT * FROM raw_pos.menu_staging;

## 2. 半構造化データとVARIANTデータ型

In [ ]:
%%sql
SELECT menu_item_health_metrics_obj FROM raw_pos.menu_staging;

In [ ]:
%%sql
-- VARIANTオブジェクトからデータを抽出
SELECT
    menu_item_name,
    CAST(menu_item_health_metrics_obj:menu_item_id AS INTEGER) AS menu_item_id,
    menu_item_health_metrics_obj:menu_item_health_metrics[0]:ingredients::ARRAY AS ingredients
FROM raw_pos.menu_staging;

In [ ]:
%%sql
-- FLATTENを使用して配列を展開
SELECT
    i.value::STRING AS ingredient_name,
    m.menu_item_health_metrics_obj:menu_item_id::INTEGER AS menu_item_id
FROM
    raw_pos.menu_staging m,
    LATERAL FLATTEN(INPUT => m.menu_item_health_metrics_obj:menu_item_health_metrics[0]:ingredients::ARRAY) i;

## 3. 動的テーブル

In [ ]:
%%sql
CREATE OR REPLACE DYNAMIC TABLE harmonized.ingredient
    LAG = '1 minute'
    WAREHOUSE = 'TB_DE_WH'
AS
    SELECT
    ingredient_name,
    menu_ids
FROM (
    SELECT DISTINCT
        i.value::STRING AS ingredient_name,
        ARRAY_AGG(m.menu_item_id) AS menu_ids
    FROM
        raw_pos.menu_staging m,
        LATERAL FLATTEN(INPUT => menu_item_health_metrics_obj:menu_item_health_metrics[0]:ingredients::ARRAY) i
    GROUP BY i.value::STRING
);

In [ ]:
%%sql
-- ingredient動的テーブルの確認
SELECT * FROM harmonized.ingredient;

## 4. 新しいメニュー項目の追加

In [ ]:
%%sql
-- バインミーサンドイッチを追加
INSERT INTO raw_pos.menu_staging 
SELECT 
    10101,
    15,
    'Sandwiches',
    'Better Off Bread',
    157,
    'Banh Mi',
    'Main',
    'Cold Option',
    9.0,
    12.0,
    PARSE_JSON('{
      "menu_item_health_metrics": [
        {
          "ingredients": [
            "French Baguette",
            "Mayonnaise",
            "Pickled Daikon",
            "Cucumber",
            "Pork Belly"
          ],
          "is_dairy_free_flag": "N",
          "is_gluten_free_flag": "N",
          "is_healthy_flag": "Y",
          "is_nut_free_flag": "Y"
        }
      ],
      "menu_item_id": 157
    }'
);

In [ ]:
%%sql
-- 新しい原材料の確認（動的テーブルの更新を待つ）
SELECT * FROM harmonized.ingredient 
WHERE ingredient_name IN ('French Baguette', 'Pickled Daikon');

## クリーンアップ

In [ ]:
%%sql
USE ROLE accountadmin;
DROP TABLE IF EXISTS raw_pos.menu_staging;
DROP TABLE IF EXISTS harmonized.ingredient;
ALTER WAREHOUSE tb_de_wh SUSPEND;